# VTK

In [ ]:
import math 
import vtk

# Let’s make a sixth-order tetrahedron 
order = 2
# The number of points for a sixth-order tetrahedron is 
nPoints = round((order + 1) * (order + 2) * (order + 3) / 6)

# Create a tetrahedron and set its number of points. Internally, Lagrange cells
 # compute their order according to the number of points they hold. 
tet = vtk.vtkLagrangeTetra() 
tet.GetPointIds().SetNumberOfIds(nPoints) 
tet.GetPoints().SetNumberOfPoints(nPoints)
tet.Initialize()

point = [0.,0.,0.]
barycentricIndex = [0, 0, 0, 0]

In [ ]:
# For each point in the tetrahedron...
for i in range(nPoints):
    # ...we set its id to be equal to its index in the internal point array. 
    tet.GetPointIds().SetId(i, i)

    # We compute the barycentric index of the point... 
    tet.ToBarycentricIndex(i, barycentricIndex)

    print(barycentricIndex)
    # ...and scale it to unity.
    for j in range(3):
        point[j] = float(barycentricIndex[j]) / order

    # A tetrahedron comprised of the above-defined points has straight
    # edges.
    tet.GetPoints().SetPoint(i, point[0], point[1], point[2])

In [ ]:
# Add the tetrahedron to a cell array 
tets = vtk.vtkCellArray() 
tets.InsertNextCell(tet)
 
# Add the points and tetrahedron to an unstructured grid 
uGrid =vtk.vtkUnstructuredGrid() 
uGrid.SetPoints(tet.GetPoints())
uGrid.InsertNextCell(tet.GetCellType(), tet.GetPointIds())

# Visualize
mapper = vtk.vtkDataSetMapper() 
mapper.SetInputData(uGrid)

actor = vtk.vtkActor() 
actor.SetMapper(mapper)

renderer = vtk.vtkRenderer() 
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# Define the timer callback to close the window after 5 seconds
class TimerCallback:
    def __init__(self, render_window_interactor):
        self.render_window_interactor = render_window_interactor
        self.timer_count = 0

    def execute(self, obj, event):
        self.timer_count += 1
        if self.timer_count >= 3:
            print("Closing window after 3 seconds")
            self.render_window_interactor.GetRenderWindow().Finalize()
            self.render_window_interactor.TerminateApp()
        else:
            print(f"Timer event {self.timer_count}")

# Set up the callback
callback = TimerCallback(render_window_interactor)
render_window_interactor.AddObserver("TimerEvent", callback.execute)

# Start the timer with an interval of 1 second (1000 ms)
render_window_interactor.CreateRepeatingTimer(1000)

renderer.AddActor(actor) 
renderer.SetBackground(.2, .3, .4)

render_window.Render()
render_window_interactor.Start()